<a href="https://colab.research.google.com/github/nupur710/Machine-Learning-in-Python/blob/master/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install scipy
!pip install scikit-learn
!pip install matplotlib
!pip install pandas
!pip install quandl

In [2]:
import pandas as pd
import quandl
import math
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

In [3]:
df= quandl.get('WIKI/GOOGL')
print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [4]:
df= df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]

In [5]:
print(df.head())

            Adj. Open  Adj. High   Adj. Low  Adj. Close  Adj. Volume
Date                                                                
2004-08-19  50.159839  52.191109  48.128568   50.322842   44659000.0
2004-08-20  50.661387  54.708881  50.405597   54.322689   22834300.0
2004-08-23  55.551482  56.915693  54.693835   54.869377   18256100.0
2004-08-24  55.792225  55.972783  51.945350   52.597363   15247300.0
2004-08-25  52.542193  54.167209  52.100830   53.164113    9188600.0


In [6]:
df['High PCT']= (df['Adj. High'] - df['Adj. Low'])/df['Adj. Low'] * 100.0

In [7]:
df['PCT Change']= (df['Adj. Close'] - df['Adj. Open'])/df['Adj. Open'] * 100.0

In [8]:
#features
df=  df[['Adj. Close', 'High PCT', 'PCT Change', 'Adj. Volume']]

In [9]:
print(df.head())

            Adj. Close  High PCT  PCT Change  Adj. Volume
Date                                                     
2004-08-19   50.322842  8.441017    0.324968   44659000.0
2004-08-20   54.322689  8.537313    7.227007   22834300.0
2004-08-23   54.869377  4.062357   -1.227880   18256100.0
2004-08-24   52.597363  7.753210   -5.726357   15247300.0
2004-08-25   53.164113  3.966115    1.183658    9188600.0


In [16]:
df

,Adj. Close,High PCT,PCT Change,Adj. Volume
Date,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0
2004-08-20,54.322689,8.537313,7.227007,22834300.0
2004-08-23,54.869377,4.062357,-1.227880,18256100.0
2004-08-24,52.597363,7.753210,-5.726357,15247300.0
2004-08-25,53.164113,3.966115,1.183658,9188600.0
...,...,...,...,...
2018-03-21,1094.000000,1.976619,0.130884,1990515.0
2018-03-22,1053.150000,3.265882,-2.487014,3418154.0
2018-03-23,1026.550000,4.089299,-2.360729,2413517.0


In [10]:
df.shape

(3424, 4)

In [11]:
df.fillna(value= 9999, inplace= True)

<ipython-input-11-19732c309d81>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(value= 9999, inplace= True)


In [12]:
#label
forecast_col= 'Adj. Close'

In [14]:
#predict out 10% of dataset
forecast_out = math.ceil(0.01 * len(df)) #shift each Adj. Close by 35 rows
forecast_out

35

In [15]:
df

,Adj. Close,High PCT,PCT Change,Adj. Volume
Date,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0
2004-08-20,54.322689,8.537313,7.227007,22834300.0
2004-08-23,54.869377,4.062357,-1.227880,18256100.0
2004-08-24,52.597363,7.753210,-5.726357,15247300.0
2004-08-25,53.164113,3.966115,1.183658,9188600.0
...,...,...,...,...
2018-03-21,1094.000000,1.976619,0.130884,1990515.0
2018-03-22,1053.150000,3.265882,-2.487014,3418154.0
2018-03-23,1026.550000,4.089299,-2.360729,2413517.0


In [23]:
#shifting so that the label column for each row is the prediction of Adj_Close forecast_out days in the future
df['label']= df[forecast_col].shift(-forecast_out)

In [18]:
df

,Adj. Close,High PCT,PCT Change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0,69.078238
2004-08-20,54.322689,8.537313,7.227007,22834300.0,67.839414
2004-08-23,54.869377,4.062357,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,7.753210,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,3.966115,1.183658,9188600.0,71.219849
...,...,...,...,...,...
2018-03-21,1094.000000,1.976619,0.130884,1990515.0,NaN
2018-03-22,1053.150000,3.265882,-2.487014,3418154.0,NaN
2018-03-23,1026.550000,4.089299,-2.360729,2413517.0,NaN


In [37]:
df.dropna(inplace=True)
df

,Adj. Close,High PCT,PCT Change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0,69.078238
2004-08-20,54.322689,8.537313,7.227007,22834300.0,67.839414
2004-08-23,54.869377,4.062357,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,7.753210,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,3.966115,1.183658,9188600.0,71.219849
...,...,...,...,...,...
2017-12-07,1044.570000,1.309689,0.820408,1437448.0,1177.370000
2017-12-08,1049.380000,1.009695,-0.231030,1479665.0,1182.220000
2017-12-11,1051.970000,1.137800,0.081818,1096997.0,1181.590000


In [38]:
X = np.array(df.drop('label', axis=1))
y= np.array(df['label'])

In [39]:
print(X)

[[ 5.03228418e+01  8.44101709e+00  3.24967503e-01  4.46590000e+07]
 [ 5.43226889e+01  8.53731343e+00  7.22700723e+00  2.28343000e+07]
 [ 5.48693765e+01  4.06235672e+00 -1.22788010e+00  1.82561000e+07]
 ...
 [ 1.05197000e+03  1.13780025e+00  8.18182683e-02  1.09699700e+06]
 [ 1.04877000e+03  1.68729124e+00 -1.17142857e-01  1.68497700e+06]
 [ 1.05139000e+03  8.50388886e-01 -6.55843662e-02  1.36958000e+06]]


In [40]:
print(y)

[  69.0782379    67.83941377   68.91272699 ... 1181.59       1119.2
 1068.76      ]


In [41]:
#features in ml in range -1 to 1, speeds up processing
X= preprocessing.scale(X)

In [42]:
#train-test split
X_train, X_test, y_train, y_test= model_selection.train_test_split(X, y, test_size= 0.2)

In [43]:
#classifier- support vector regression
clf= svm.SVR()

In [44]:
#train-- fit train features with labels
clf.fit(X_train, y_train)

SVR()

In [45]:
#test on test features and labels
confidence= clf.score(X_test, y_test)

In [46]:
print(confidence)

0.7858405238944479


In [49]:
#linear regression is a multi-threaded algo
clf= LinearRegression(n_jobs= 1)

In [50]:
clf.fit(X_train, y_train)
confidence= clf.score(X_test, y_test)
print(confidence)

0.9769435431124281


In [52]:
#confidence score with different kernels
for k in ['linear', 'poly', 'rbf', 'sigmoid']:
  clf= svm.SVR(kernel= k)
  clf.fit(X_train, y_train)
  confidence= clf.score(X_test, y_test)
  print(k, confidence)

linear 0.9757660645543863
poly 0.7217805657367704
rbf 0.7858405238944479
sigmoid 0.8845632953484226


In [ ]:
#linear regression classifier performs best
clf= LinearRegression()
clf.fit(X_train, y_train)

In [55]:
X_lately= X[-forecast_out:]
X_lately

array([[ 2.47789375e+00, -6.26371563e-01,  1.49673910e-01,
        -7.87180204e-01],
       [ 2.49028416e+00, -3.08520353e-01,  3.65676033e-01,
        -7.93000819e-01],
       [ 2.49011728e+00, -3.70012321e-01, -4.54598528e-01,
        -7.37542033e-01],
       [ 2.66637810e+00,  9.78552730e-01,  1.88428310e-01,
        -3.37894297e-01],
       [ 2.66412530e+00, -3.06522464e-01,  2.72170876e-01,
        -6.87147221e-01],
       [ 2.66374984e+00, -4.98654371e-01,  1.79758877e-02,
        -7.78206098e-01],
       [ 2.70363276e+00, -5.61842639e-01,  4.18771228e-01,
        -7.03993714e-01],
       [ 2.70517634e+00, -3.56541713e-01,  2.06132326e-01,
        -8.09254404e-01],
       [ 2.73446276e+00, -6.21405278e-01,  4.77565734e-01,
        -7.92659118e-01],
       [ 2.70396651e+00, -7.84683356e-01, -3.91943483e-01,
        -8.49727107e-01],
       [ 2.74447521e+00, -7.97218899e-01,  1.89157234e-01,
        -8.06644351e-01],
       [ 2.72499265e+00, -4.85829899e-01, -2.38595936e-03,
      